In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark
!pip install faiss-cpu --no-cache
!pip install autofaiss

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 45 kB/s 
     |████████████████████████████████| 199 kB 49.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845514 sha256=67a9634a3c93158a86516e00fd9ce594d16ba4d722a6d576fbf1072e0866f515
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.6 MB 5.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 3.3 MB/s 
     |████████████████████████████████| 87 kB 4.6 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115940 sha256=a1568878834dcc860709689d13855814eb4926

In [ ]:
import numpy as np
import pandas as pd
import sys
import time
import os
import shutil
import faiss
import glob
import numpy as np
from pyspark.sql import SparkSession

import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from collections import Counter
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import time
from operator import add
from sklearn.neighbors import KNeighborsClassifier

dataset


In [ ]:
from sklearn.datasets import fetch_covtype
forest = fetch_covtype()
Data= forest['data']
label = forest['target']
data_cov = np.concatenate([Data[:, :32], label.reshape(-1, 1)], axis = -1)

In [ ]:
data_cov = data_cov[:10000, :]
data_cov.shape

(10000, 33)

In [ ]:
all_cov, query_cov = train_test_split(data_cov, test_size=0.2, random_state=4)

In [ ]:
# from keras.datasets import mnist
# (train_X, train_y), (test_X, test_y) = mnist.load_data()
# data_p1 = np.concatenate([train_X.reshape(-1, 28 * 28), train_y.reshape(-1, 1)], axis = -1)
# data_p2 = np.concatenate([test_X.reshape(-1, 28 * 28), test_y.reshape(-1, 1)], axis = -1)
# data_mnist = np.concatenate([data_p1, data_p2], axis = 0)

In [ ]:
# data_mnist = data_mnist[:10000, :]
# data_mnist.shape

In [ ]:
# all_mnt, query_mnt = train_test_split(data_mnist, test_size=0.2, random_state=4)

naive baseline

In [ ]:
def calculateDistance(test_point, train_point, p=2):
    # minkowski distance
    # p = 2 --> euclidean distance
    
    dist = np.sum((test_point[:-1]-train_point[:-1])**p)
    dist = dist**(1/p)
    return dist

def calculateDistance2(test_point,train_point):
    # Euclidean distance
    dist = np.sqrt(np.sum((test_point-train_point)**2))
    return dist

def origKNN(train_x, test_x, k, p=2):
    
    start = time.time()


    train_y = train_x[:, -1].copy()
    test_y = test_x[:, -1].copy()

    predList = []
    count_ = 0
    for test_point in test_x:
        distList = []
        for train_point in train_x:
            distance = calculateDistance(test_point,train_point, p)
            distList.append((distance, train_point[-1]))
        distList = sorted(distList)[:k]
        tmp = []
        for dis, label in distList:
          tmp.append(label)
        count = Counter(tmp)
        finalPrediction = count.most_common(1)[0][0]
        predList.append(finalPrediction) 
        count_ += 1
        if count_ % 1000 == 0: print('count_ = ', count_)
      
    print('count_ = ', count_)
    end = time.time()

    print("Original KNN running time: {}".format(end-start))
    print(accuracy_score(test_y, predList))

In [ ]:
for k in [8, 16, 32, 64]:
  origKNN(all_cov, query_cov, k)

count_ =  1000
count_ =  2000
count_ =  2000
Original KNN running time: 173.9551968574524
0.7785
count_ =  1000
count_ =  2000
count_ =  2000
Original KNN running time: 175.52844309806824
0.747
count_ =  1000
count_ =  2000
count_ =  2000
Original KNN running time: 174.9058539867401
0.7115
count_ =  1000
count_ =  2000
count_ =  2000
Original KNN running time: 174.92647242546082
0.6535
